In [10]:
import pandas as pd
import numpy as np
import sys

In [11]:
interactions = pd.read_csv('data/interactions.tsv',sep='\t',header=None)

new_header = interactions.iloc[0] 
interactions = interactions[1:]
interactions.columns = new_header

interactions

C:\Users\Admin\AppData\Local\Programs\Python\Python39\lib\site-packages\IPython\core\interactiveshell.py:3441: DtypeWarning: Columns (2,9) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,gene_name,gene_claim_name,entrez_id,interaction_claim_source,interaction_types,drug_claim_name,drug_claim_primary_name,drug_name,drug_concept_id,interaction_group_score,PMIDs
1,CDK7,CDK7,1022,CancerCommons,inhibitor,SNS-032,SNS-032,BMS-387032,chembl:CHEMBL296468,0.82,NaN
2,VDR,VDR,7421,DTC,NaN,NIFEKALANT,NIFEKALANT,NIFEKALANT,chembl:CHEMBL360861,0.14,NaN
3,GPR55,GPR55,9290,DTC,NaN,BISACODYL,BISACODYL,BISACODYL,chembl:CHEMBL942,0.94,NaN
4,NR1H4,NR1H4,9971,DTC,NaN,PENTACHLOROPHENOL,PENTACHLOROPHENOL,PENTACHLOROPHENOL,chembl:CHEMBL75967,0.07,NaN
5,FGFR2,FGFR2,2263,JAX-CKB,NaN,AZ6089,AZ6089,NaN,NaN,NaN,22869148
...,...,...,...,...,...,...,...,...,...,...,...
85456,PIK3C3,PIK3C3,5289.0,MyCancerGenome,inhibitor,BGJ398,BGJ398,INFIGRATINIB,chembl:CHEMBL1852688,0.17,NaN
85457,ABL1,Tyrosine-protein kinase ABL1,25.0,TTD,NaN,Azaindole derivative 2,Azaindole derivative 2,NaN,NaN,NaN,NaN
85458,CACNA1D,776,776.0,GuideToPharmacology,inhibitor,135651166,[3H](+)-ISRADIPINE,NaN,NaN,NaN,NaN
85459,PRKCA,PRKCA,5578.0,DTC,NaN,RESVERATROL,RESVERATROL,RESVERATROL,chembl:CHEMBL165,0.12,21880495


In [13]:
interactions['interaction_types'].isnull().sum()

62428

In [14]:
62428/85460 

0.7304937982681956

Drug Overlap between our data and RepoDB

In [2]:
repodb_df = pd.read_csv('data/repo_db.csv')
repodb_df['drug_name'] = repodb_df['drug_name'].str.lower()

In [3]:
ourdata = pd.read_csv('data/drug_riskscores.csv')
ourdata['drug'] = ourdata['drug'].str.lower()

In [4]:
drug_overlap_repodb = ourdata.join(repodb_df.set_index(['drug_name']), how='inner', on=['drug'], lsuffix='_x', rsuffix='_y')
drug_overlap_repodb = drug_overlap_repodb.iloc[: , 1:]
#drug_overlap_repodb.to_csv('drug_overlap_repodb.csv')
drug_overlap_repodb

,drug,risk_score,drug_id,ind_name,ind_id,NCT,status,phase,DetailedStatus
15,abarelix,0.189733,DB00106,Ischemic priapism,C2711256,NaN,Approved,NaN,NaN
16,anidulafungin,0.132178,DB00362,Candidiasis of the esophagus,C0239295,NaN,Approved,NaN,NaN
16,anidulafungin,0.132178,DB00362,candidal peritonitis,C2062982,NaN,Approved,NaN,NaN
16,anidulafungin,0.132178,DB00362,Candidemia,C0877445,NaN,Approved,NaN,NaN
16,anidulafungin,0.132178,DB00362,Fungemia,C0085082,NCT00806351,Terminated,Phase 3,The study was prematurely terminated on May 18...
...,...,...,...,...,...,...,...,...,...
807,zonisamide,0.039542,DB00909,Epilepsy,C0014544,NCT00692003,Terminated,Phase 3,Sponsor's decision
807,zonisamide,0.039542,DB00909,"Alcoholic Intoxication, Chronic",C0001973,NCT00862563,Terminated,Phase 2,Recruitment goals could not be met before endi...
807,zonisamide,0.039542,DB00909,alcohol use disorder,C0001956,NCT01566370,Terminated,Phase 2,recruitment infeasible
807,zonisamide,0.039542,DB00909,Bipolar Disorder,C0005586,NCT01566370,Terminated,Phase 2,recruitment infeasible


Drug Overlap between our data and BROAD

In [5]:
broad_df = pd.read_csv('data/broad_repurp_data_orig.tsv',sep='\t',header=None)
broad_df = broad_df.drop(range(9))

new_header = broad_df.iloc[0] 
broad_df = broad_df[1:]
broad_df.columns = new_header

broad_df['pert_iname'] = broad_df['pert_iname'].str.lower()

In [6]:
drug_overlap_broad = ourdata.join(broad_df.set_index(['pert_iname']), how='inner', on=['drug'], lsuffix='_x', rsuffix='_y')
drug_overlap_broad = drug_overlap_broad.iloc[: , 1:]
#drug_overlap_broad.to_csv('drug_overlap_broad.csv')
drug_overlap_broad

,drug,risk_score,clinical_phase,moa,target,disease_area,indication
15,abarelix,0.189733,Phase 3,gonadotropin releasing factor hormone receptor...,GNRHR,NaN,NaN
16,anidulafungin,0.132178,Launched,"fungal 1,3-beta-D-glucan synthase inhibitor",NaN,infectious disease,candidemia|fungal infection|esophageal candidi...
19,bivalirudin,0.039756,Launched,thrombin inhibitor,F2,cardiology,angina pectoris
23,carfilzomib,0.076173,Launched,proteasome inhibitor,PSMA1|PSMA2|PSMA3|PSMA4|PSMA5|PSMA6|PSMA7|PSMA...,hematologic malignancy,multiple myeloma
28,eptifibatide,0.013718,Launched,platelet aggregation inhibitor,ITGA2B|ITGB3,cardiology,acute coronary syndrome (ACS)|percutaneous cor...
...,...,...,...,...,...,...,...
800,zidovudine,0.293031,Launched,nucleoside reverse transcriptase inhibitor,TERT,infectious disease,human immunodeficiency virus (HIV-1)|acquired ...
802,zileuton,0.087954,Launched,leukotriene synthesis inhibitor|lipoxygenase i...,ALOX5,pulmonary,asthma
803,ziprasidone,0.020284,Launched,dopamine receptor antagonist|serotonin recepto...,ADRA1A|ADRA1B|ADRA2A|ADRA2B|ADRA2C|CHRM1|CHRM2...,neurology/psychiatry,schizophrenia|bipolar disorder
805,zolmitriptan,0.021104,Launched,serotonin receptor agonist,HTR1A|HTR1B|HTR1D|HTR1E|HTR1F,neurology/psychiatry,migraine headache
